In [1]:
print("hello")
# Go here https://python.langchain.com/v0.1/docs/get_started/quickstart/ and use the web scraping stuff to pass in the Vice article

hello


In [4]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.documents import Document
from langchain.chains import create_retrieval_chain
from bs4 import BeautifulSoup
import requests


In [5]:
#%% User Input
# let's try reading in a scientific paper 
url = "https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7871999/"

In [6]:
#%% Parse the webpage and find all the external links
# Get the webpage content
response = requests.get(url)

#https://letscodepython.com/blog/2017/11/10/scraping-web-beautiful-soup/

# Parse the HTML content
soup = BeautifulSoup(response.content, 'html.parser')



In [7]:
# Get the web page text
embeddings = OpenAIEmbeddings()
loader = WebBaseLoader(url) # Is there another option like this that can include links?
docs = loader.load()

# Ingest web page into a vectorstore to feed to the llm
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

# I 


In [8]:

llm = ChatOpenAI()

prompt = ChatPromptTemplate.from_template("""Answer the following question by referencing the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

#
response = retrieval_chain.invoke({"input": "Explain 3 takeaways from the provided scientific article."})
print(response["answer"])




1. The study focused on individual-specific computational modeling of rat optic nerve head (ONH) biomechanics, incorporating a fiber-reinforced model for the sclera. This approach improved upon past models that assumed a simple neo-Hookean solid for the sclera.

2. The results of the study showed that elevated intraocular pressure (IOP) led to increased stress and strain in the rat ONH, with the highest strains occurring primarily in the inferior nerve. This pattern was consistent across different ONH models and variants, highlighting the importance of scleral collagen fiber direction on optic nerve strains.

3. The study provided insights into how biomechanical insult affects retinal ganglion cell (RGC) pathogenesis in glaucoma, with a focus on understanding the biomechanical environment of the ONH and the cellular responses that occur during the disease. By simulating the effects of elevated IOP in rat ONH models, the research aimed to inform future modeling studies and improve the u

In [9]:
# Let's see if the llm can identify an obviously incorrect statement
response = retrieval_chain.invoke({"input": "Does the provided scientific article support this statement, 'Elevated IOP leads to increased strain, primarily in the superior nerve, of the rat ONH.'?"})
print(response["answer"])

No, the provided scientific article does not support the statement that elevated IOP leads to increased strain primarily in the superior nerve of the rat ONH. The article states that the highest strains were seen inferiorly in the rat optic nerve, and this pattern was consistent across different model geometries and variants. It also mentions that RGC axonal degeneration occurs preferentially in the superior nerve, while astrocytes are activated in the inferior nerve before the superior nerve. However, the article does not support the claim that elevated IOP primarily leads to increased strain in the superior nerve of the rat ONH.


In [10]:
# Ok let's see if the llm can determine if the work in the rat ONH paper backs up a statement. I'm going to feed it a statement that is supported by
# a paper referenced in the rat ONH paper, not the rat ONH paper itself
response = retrieval_chain.invoke({"input": "Does the work described in the provided scientific article support this statement, 'Elevated IOP causes astrocytes in the inferior rat ONH to reorient before those in the superior ONH'?"})
print(response["answer"])
# The llm concluded "yes" -- technically not correct

Yes, the work described in the scientific article supports the statement that elevated IOP causes astrocytes in the inferior rat ONH to reorient before those in the superior ONH. The study found that astrocytes in the inferior nerve withdraw processes extending into the superior nerve, leaving the superior axons more vulnerable to apoptosis. This finding is based on the observation of strain patterns in the rat ONH models, where astrocyte activation and reorientation were seen in the inferior nerve before those in the superior nerve. Further studies will be needed to validate this hypothesis.


In [11]:
# Let's ask a more specific question
response = retrieval_chain.invoke({"input": "Does the work described in the provided scientific article support this statement, 'Elevated IOP causes astrocytes in the inferior rat ONH to reorient before those in the superior ONH'? Or does it simply cite another paper to support that statement?"})
print(response["answer"])
# The llm didn't really answer the way I'd like it to. Maybe I'm being unfair, or maybe we'd need a more sophisticated model (like gpt4) to answer this

The work described in the provided scientific article does support the statement that elevated IOP causes astrocytes in the inferior rat ONH to reorient before those in the superior ONH. The study presents data that aligns with this hypothesis, indicating that astrocytes in the inferior nerve withdraw processes extending into the superior nerve, leaving the superior axons more vulnerable to apoptosis. This finding is based on the strain patterns observed in the rat ONH models studied, suggesting a potential mechanism for the preferential degeneration of RGC axons in the superior nerve.


In [12]:
# Let's take a leap in logic. We'll ask something that is not really supported by the paper.
response = retrieval_chain.invoke({"input": "Does the work described in the provided scientific article support this statement, 'Glaucoma can be cured by reducing astrocyte reactivity to elevated IOP'?"})
print(response["answer"])
# LLM provided a pretty good response here

No, the work described in the scientific article does not support the statement that 'Glaucoma can be cured by reducing astrocyte reactivity to elevated IOP.' The article focuses on studying the biomechanical environment of the optic nerve head in relation to glaucoma, specifically looking at the effects of elevated IOP on retinal ganglion cells. The study does not mention astrocyte reactivity or suggest that reducing it could cure glaucoma.


In [13]:
# Let's ask a tough question
response = retrieval_chain.invoke({"input": "Does the work carried out in the provided scientific article support this statement, 'Biomechanics are an important driver in glaucoma'?"})
print(response["answer"])
# The LLM was fooled here. The article cites research to back up the claim that biomechanics are important in glaucoma, but the actual work carried out
# in the article does not support this

Yes, the work carried out in the provided scientific article supports the statement that biomechanics are an important driver in glaucoma. The article discusses how elevated intraocular pressure (IOP) is a key risk factor for glaucoma and how biomechanical insult to the optic nerve head (ONH) can lead to tissue damage and remodeling, as well as alterations in cell behavior within the ONH. The study used finite element modeling to analyze ONH models with individual-specific geometry and found that changing the collagen fiber direction and scleral stiffness resulted in changes in strain within the optic nerve, which is consistent with the idea that biomechanics play a role in RGC pathogenesis in glaucoma.


In [ ]:
# Perhaps I was being a bit too nitpicky or expecting too much of the llm. Perhaps the types of statements that we'd need to verify
# won't be so tricky. We can also make the llm/agent cite the portions of text it uses to answer the question. This might help
# determine if the statement is actually backed up by the research performed within a given article or if the statement is supported by citations
# or is simply an assertion in the article intro. etc.